In [17]:
import flask
import joblib
from flask import Flask, jsonify, request
import pandas as pd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import re
import nltk
nltk.download('stopwords')
nltk.download('rslp')
import random
import seaborn as sns
from sklearn.cluster import KMeans
#from sklearn.externals import joblib
import matplotlib.pyplot as plt
#import pyplot
from scipy.stats import randint as sp_randint
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from scipy.sparse import hstack
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss,accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, AdaBoostClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [20]:
def single_data(df):
   df_olist_orders_data=pd.read_excel(df,sheet_name='olist_orders_data')
  
   df_olist_orders_data.drop(['Unnamed: 0'],axis=1,inplace=True)  

   df_olist_order_reviews_data=pd.read_excel(df,sheet_name='olist_order_reviews_dataset')
  
   df_olist_order_reviews_data.drop(['Unnamed: 0'],axis=1,inplace=True)
  
   df_olist_customers_data=pd.read_excel(df,sheet_name='olist_customers_data')
  
   df_olist_customers_data.drop(['Unnamed: 0'],axis=1,inplace=True)
  
   df_olist_order_items_dataset=pd.read_excel(df,sheet_name='olist_order_items_dataset')
  
   #df_olist_order_items_dataset.drop(['Unnamed: 0'],axis=1,inplace=True)
  
   df_olist_products_dataset=pd.read_excel(df,sheet_name='olist_products_dataset')
  
   df_olist_products_dataset.drop(['Unnamed: 0'],axis=1,inplace=True)
  
   df_olist_order_payments_dataset=pd.read_excel(df,sheet_name='olist_order_payments_dataset')
  
   df_olist_order_payments_dataset.drop(['Unnamed: 0'],axis=1,inplace=True)
  
   df_product_category_name=pd.read_excel(df,sheet_name='product_category_name')
  
   df_product_category_name.drop(['Unnamed: 0'],axis=1,inplace=True)

   df_olist_order_reviews_data=df_olist_order_reviews_data[['order_id','review_score','review_comment_message']]
   
   df_order_review=df_olist_order_reviews_data.merge(df_olist_orders_data,on='order_id')

   df_product_english=pd.merge(df_olist_products_dataset,df_product_category_name,on='product_category_name',how='left')

   df_product_english=df_product_english.drop(labels='product_category_name',axis=1)
   
   df_product_item=pd.merge(df_olist_order_items_dataset,df_product_english,on='product_id')
   
   df_product_review=pd.merge(df_order_review,df_product_item,on='order_id')
   
   df_product_review_payments=pd.merge(df_product_review,df_olist_order_payments_dataset,on='order_id')
   
   data_final_predict=pd.merge(df_product_review_payments,df_olist_customers_data,on='customer_id')

   data_final_predict['review_score']=data_final_predict['review_score'].apply(lambda x:1 if x>3 else 0)

   list=['product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm']
  
   res={'product_name_lenght': 52.0,'product_description_lenght': 600.0,'product_photos_qty': 1.0,'product_weight_g': 700.0,'product_length_cm': 25.0,'product_height_cm': 13.0,'product_width_cm': 20.0}

   for i in list:
      if data_final_predict[i] is None:
          data_final_predict[i].fillna(res.get(i),inplace=True)

   ids=data_final_predict[data_final_predict['order_delivered_customer_date'].isnull() == True].index.values
   vals=data_final_predict.iloc[ids]['order_estimated_delivery_date'].values
   data_final_predict.loc[ids,'order_delivered_customer_date']=vals
   ids=data_final_predict[data_final_predict['order_approved_at'].isnull() == True].index.values
   data_final_predict.loc[ids,'order_approved_at']=data_final_predict.iloc[ids]['order_purchase_timestamp'].values
   data_final_predict.drop(labels='order_delivered_carrier_date',axis=1,inplace=True)
   data_final_predict['review_comment_message']=data_final_predict['review_comment_message'].fillna('no_review') 
   
   
   # converting date to datetime and extracting dates from the datetime columns in the data set
   
   datetime_columns = ['order_purchase_timestamp','order_approved_at','order_delivered_customer_date','order_estimated_delivery_date']
   for columns in datetime_columns:
      data_final_predict[columns] = pd.to_datetime(data_final_predict[columns].dt.date)
    
    # https://www.kaggle.com/andresionek/predicting-customer-satisfaction
   # calculating estimated delivery time
   data_final_predict['estimated_delivery_time'] = (data_final_predict['order_estimated_delivery_date'] - data_final_predict['order_approved_at']).dt.days

   # calculating actual delivery time
   data_final_predict['actual_delivery_time'] = (data_final_predict['order_delivered_customer_date'] - data_final_predict['order_approved_at']).dt.days

   # calculating diff_in_delivery_time
   data_final_predict['diff_in_delivery_time'] = data_final_predict['estimated_delivery_time'] - data_final_predict['actual_delivery_time']

   # finding if delivery was lare
   data_final_predict['on_time_delivery'] = data_final_predict['order_delivered_customer_date'] < data_final_predict['order_estimated_delivery_date']
   data_final_predict['on_time_delivery'] = data_final_predict['on_time_delivery'].astype('int')

   # finding total order cost
   data_final_predict['total_order_cost'] = data_final_predict['price'] + data_final_predict['freight_value']

   # calculating order freight ratio
   data_final_predict['order_freight_ratio'] = data_final_predict['freight_value']/data_final_predict['price']

   # finding the day of week on which order was made
   data_final_predict['purchase_dayofweek'] = pd.to_datetime(data_final_predict['order_purchase_timestamp']).dt.dayofweek

   # adding is_reviewed where 1 is if review comment is given otherwise 0.
   data_final_predict['is_reviewed'] = (data_final_predict['review_comment_message'] != 'no_review').astype('int') 

   data_final_predict.drop(columns=['order_id', 'order_item_id', 'product_id', 'seller_id','shipping_limit_date','customer_id',
                       'order_purchase_timestamp', 'order_approved_at', 'order_delivered_customer_date', 'customer_state',
                       'order_estimated_delivery_date','customer_unique_id', 'customer_city','customer_zip_code_prefix'],
              axis=1,inplace=True)
   # numerical features
   numerical = ['price', 'freight_value', 'product_name_lenght','product_description_lenght', 'product_photos_qty','product_weight_g','product_length_cm', 'product_height_cm', 'product_width_cm', 'payment_sequential','payment_installments', 'payment_value','on_time_delivery', 'estimated_delivery_time','actual_delivery_time', 'diff_in_delivery_time', 'purchase_dayofweek','total_order_cost', 'order_freight_ratio','is_reviewed']
  
   # categorical features
   categorical = ['review_comment_message','product_category_name_english','order_status', 'payment_type']
   
   def clean_data(data):
    
    #list_text = []
    
    portuguese_stop = stopwords.words('portuguese') # portugese language stopwords
    #stemmer_1 = RSLPStemmer() # portugese language stemmer
    
    links_alphabet = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' # check for hyperlinks
    numerical_dates = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)\d{2,4}' # check for dates
    currency_symbols = '[R]{0,1}\$[ ]{0,}\d+(,|\.)\d+' # check for currency symbols
    
    if data is None:
        return ''
    else:
        if isinstance(data,str):
            data = re.sub('\n', ' ', data) # remove new lines
            data = re.sub(links_alphabet, ' URL ', data) # remove hyperlinks
            data= re.sub(numerical_dates, ' ', data) # remove dates
            data = re.sub(currency_symbols, ' dinheiro ', data) # remove currency symbols
            data = re.sub('[0-9]+', ' numero ', data) # remove digits
            data = re.sub('([nN][ãÃaA][oO]|[ñÑ]| [nN] )', ' negação ', data) # replace no with negative
            data = re.sub('\W', ' ', data) # remove extra whitespaces
            data = re.sub('\s+', ' ', data) # remove extra spaces
            data = re.sub('[ \t]+$', '', data) # remove tabs etc.
            data = ' '.join(e for e in data.split() if e.lower() not in portuguese_stop) # remove stopwords
            #text = ' '.join(stemmer.stem(e.lower()) for e in text.split()) # stemming the words
            #list_text.append(data.lower().strip())
        
    return data
   
   clean_process = clean_data(data_final_predict['review_comment_message'].values)

   data_final_predict['review_comment_message'] = clean_process
   # nao_reveja = no_review in portugese
   data_final_predict['review_comment_message'] = data_final_predict['review_comment_message'].replace({'no_review':'nao_reveja'}) 
   # df_final.to_csv('olist_final.csv',index=False)
  
   # Encoding categorical variable payment_type
   data_final_predict['payment_type'] = data_final_predict['payment_type'].replace({'credit_card':1,'boleto':2,'voucher':3,'debit_card':4})
 
   data_final_predict.drop(['payment_value'],axis=1,inplace=True)
   data_final_predict.drop(['review_score'],axis=1,inplace=True)

   fhs_rf_model=joblib.load('/content/random_forest.pkl')
   product_category_name_english_1=joblib.load('/content/product_category2.pkl')
   review_comment_message_1=joblib.load('/content/review_comment2.pkl')
   order_status_1=joblib.load('/content/order_status2.pkl')

   order_status_transform=order_status_1.transform(data_final_predict.order_status.values)
   product_category_name_english_transform=product_category_name_english_1.transform(data_final_predict.product_category_name_english.values)
   review_comment_message_1_transform=review_comment_message_1.transform(data_final_predict.review_comment_message.values)
   
   data_final_predict = data_final_predict.drop(labels=['review_comment_message','product_category_name_english','order_status'],axis=1)

   data_final_concat=hstack((data_final_predict,order_status_transform,product_category_name_english_transform,review_comment_message_1_transform))
 
   y1=fhs_rf_model.predict(data_final_concat)

   if y1 == 1:
     return 'positive'

   elif y1 == 0:
     return 'negative'  

y=single_data('/content/Book5A.xlsx')
y

'negative'

In [32]:
df=pd.read_csv('/content/Ytest.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df

,review_score
0,1
1,1
2,1
3,1
4,1
...,...
23117,1
23118,0
23119,0
23120,1


In [33]:
df.squeeze()

0        1
1        1
2        1
3        1
4        1
        ..
23117    1
23118    0
23119    0
23120    1
23121    0
Name: review_score, Length: 23122, dtype: int64

In [35]:
df1=pd.read_csv('/content/Test.csv')
df1

,Unnamed: 0,order_id,review_comment_message,order_status,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,...,payment_type,payment_installments,estimated_delivery_time,actual_delivery_time,diff_in_delivery_time,on_time_delivery,total_order_cost,order_freight_ratio,purchase_dayofweek,is_reviewed
0,605,e6b1546a7ed77ba5ca2c2077aafd3c97,nao_reveja,delivered,99.90,34.24,57.0,272.0,1.0,2500.0,...,1,1,27,11,16,1,134.14,0.342743,4,0
1,43418,b335c1b6b81809fb3de72bab88595134,nao_reveja,delivered,32.90,11.95,48.0,185.0,1.0,650.0,...,1,1,37,8,29,1,44.85,0.363222,2,0
2,62040,d65848ec24d6595e5a459c4196c01b7c,amei,delivered,59.99,25.70,41.0,1159.0,4.0,350.0,...,1,4,38,21,17,1,85.69,0.428405,0,1
3,44491,5c05ba7b96e7ea5014e1eda4953778c3,nao_reveja,delivered,139.90,18.23,39.0,1201.0,1.0,1168.0,...,1,1,26,9,17,1,158.13,0.130307,5,0
4,31881,f06f7b506ca56d1a660918f04f7814ec,otimo produto,delivered,169.90,17.44,55.0,815.0,4.0,1600.0,...,1,1,25,13,12,1,187.34,0.102649,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23117,88414,70ee631a6eb795832e5318331fdf56d3,nao_reveja,delivered,149.94,40.66,33.0,1207.0,1.0,12650.0,...,1,10,31,22,9,1,190.60,0.271175,1,0
23118,7234,e46475bd25ec0611f7ce2a2f0e63bc8f,nao_reveja,delivered,99.99,14.43,58.0,1390.0,2.0,3475.0,...,1,1,38,14,24,1,114.42,0.144314,1,0
23119,109458,1fa1a852a56406283704f49720016285,boa tarde hoje ainda negação recebi compra ten...,invoiced,21.90,21.15,43.0,538.0,1.0,150.0,...,1,1,28,28,0,0,43.05,0.965753,4,1
23120,5271,566f53fc7d36fa366f7e221468121877,nao_reveja,delivered,69.90,15.70,39.0,720.0,4.0,300.0,...,2,1,40,10,30,1,85.60,0.224607,3,0


In [40]:
def merged_data(data):
  data1=pd.read_csv(data)
  data1.drop(['Unnamed: 0'],axis=1,inplace=True)
  data1.drop(['order_id'],axis=1,inplace=True)

  ytest=pd.read_csv('/content/Ytest.csv')
  ytest.drop(['Unnamed: 0'],axis=1,inplace=True)

  fhs_rf_model=joblib.load('/content/random_forest.pkl')
  product_category_name_english_1=joblib.load('/content/product_category2.pkl')
  review_comment_message_1=joblib.load('/content/review_comment2.pkl')
  order_status_1=joblib.load('/content/order_status2.pkl')

  order_status_transform=order_status_1.transform(data1.order_status.values)
  product_category_name_english_transform=product_category_name_english_1.transform(data1.product_category_name_english.values)
  review_comment_message_1_transform=review_comment_message_1.transform(data1.review_comment_message.values.astype('U'))
 
  data1 = data1.drop(labels=['review_comment_message','product_category_name_english','order_status'],axis=1)

  data_final_concat=hstack((data1,order_status_transform,product_category_name_english_transform,review_comment_message_1_transform))
 
  y1=fhs_rf_model.predict(data_final_concat)
  
  score=f1_score(ytest.squeeze(),y1)

  return score
u=merged_data('/content/Test.csv')
u


0.9382469567146383